In [1]:
# approach the object
import rospy
from geometry_msgs.msg import Twist, PoseWithCovariance, PoseWithCovarianceStamped, Pose
import actionlib
from control_msgs.msg import FollowJointTrajectoryAction, FollowJointTrajectoryGoal
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from rail_segmentation.srv import *
from rail_manipulation_msgs.srv import *
import tf2_ros
import numpy as np
from sensor_msgs.msg import PointCloud2
import sensor_msgs.point_cloud2 as pc2
from ros_numpy import numpify, msgify
from sensor_msgs.msg import LaserScan
import math
from nav_msgs.msg import OccupancyGrid
from nav_msgs.srv import SetMap, SetMapRequest

In [2]:
rospy.init_node('command_sender', anonymous=True)
base_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
rate = rospy.Rate(10)  # Adjust the frequency as per your requirement

In [3]:
# get laser scan transform from tf tree
tfBuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfBuffer)

In [4]:
try:
    laser_trans = tfBuffer.lookup_transform('base_link', 'laser_link', rospy.Time())
    laser_pose = numpify(laser_trans.transform)
except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException):
    print("tf error")

In [5]:
def convert_pointcloud2_to_pc(pointcloud2_msg):
    pc_data = pc2.read_points(pointcloud2_msg, skip_nans=True, field_names=("x", "y", "z"))
    pc_list = []

    for p in pc_data:
        pc_list.append([p[0], p[1], p[2]])

    pc_array = np.array(pc_list, dtype=np.float32)

    return pc_array

def move_base(base_pub_, linear_vel, angular_vel):
    cmd_vel = Twist()
    cmd_vel.linear.x = linear_vel
    cmd_vel.angular.z = angular_vel
    base_pub_.publish(cmd_vel)

In [6]:
disc_size = .04
# Discretization Factor
disc_factor = 1/disc_size
# Max Lidar Range
max_lidar_range = 3
# Create Image Size Using Range and Discretization Factor
image_size = int(max_lidar_range*2*disc_factor)

In [7]:
# based on the current scan data, generate a local map
current_scan_data = rospy.wait_for_message("/base_scan", LaserScan)
# print current_scan_data

maxAngle = current_scan_data.angle_max
minAngle = current_scan_data.angle_min
angleInc = current_scan_data.angle_increment
maxLength = current_scan_data.range_max
ranges = current_scan_data.ranges
num_pts = len(ranges)
xy_scan = np.zeros((num_pts,2))
blank_image = np.zeros((image_size,image_size,3),dtype=np.uint8)

map_resolution = 0.1  # Resolution of the occupancy grid (meters per cell)
map_width = 100       # Width of the occupancy grid (number of cells)
map_height = 100  

# generate occupancy_grid map
occupancy_grid = OccupancyGrid()
occupancy_grid.header.frame_id = 'map'
occupancy_grid.info.width = map_width
occupancy_grid.info.height = map_height
occupancy_grid.info.resolution = map_resolution
occupancy_grid.info.origin.position.x = -map_width * map_resolution / 2.0
occupancy_grid.info.origin.position.y = -map_height * map_resolution / 2.0
occupancy_grid.info.origin.orientation.w = 1.0

occupancy_grid.data = np.ones(map_width * map_height, dtype=np.int8) * -1

x_indices = np.clip(np.round((current_scan_data.ranges * np.cos(current_scan_data.angle_increment * np.arange(len(current_scan_data.ranges))) - occupancy_grid.info.origin.position.x) / map_resolution).astype(np.int32), 0, map_width - 1)
y_indices = np.clip(np.round((current_scan_data.ranges * np.sin(current_scan_data.angle_increment * np.arange(len(current_scan_data.ranges))) - occupancy_grid.info.origin.position.y) / map_resolution).astype(np.int32), 0, map_height - 1)
indices = np.ravel_multi_index((x_indices, y_indices), (map_width, map_height))

# Set the occupied cells in the occupancy grid
occupancy_grid.data[indices] = 100

# Loop through all points converting distance and angle to X,Y point
for i in range(num_pts):
    # Check that distance is not longer than it should be
    if (ranges[i] > maxLength - 0.1) or (math.isnan(ranges[i])):
        pass
    else:
        # Calculate angle of point and calculate X,Y position
        angle = minAngle + float(i)*angleInc
        xy_scan[i][0] = float(ranges[i]*math.cos(angle))
        xy_scan[i][1] = float(ranges[i]*math.sin(angle))

# Loop through all points plot in blank_image
for i in range(num_pts):
    if xy_scan[i,0] == 0 and xy_scan[i,1] == 0:
        continue
    point_in_base_link = np.dot(laser_pose, np.array([xy_scan[i,0], xy_scan[i,1], 0, 1]))
    pt_x = point_in_base_link[0]
    pt_y = point_in_base_link[1]
    if (pt_x < max_lidar_range) or (pt_x > -1 * (max_lidar_range-disc_size)) or (pt_y < max_lidar_range) or (pt_y > -1 * (max_lidar_range-disc_size)):
        pix_x = int(math.floor((pt_x + max_lidar_range) * disc_factor))
        pix_y = int(math.floor((max_lidar_range - pt_y) * disc_factor))
        if (pix_x > image_size) or (pix_y > image_size):
            print "Error"
        else:
            blank_image[pix_y,pix_x] = [0,0,255]

# show the robot point
pt_x = 0
pt_y = 0
if (pt_x < max_lidar_range) or (pt_x > -1 * (max_lidar_range-disc_size)) or (pt_y < max_lidar_range) or (pt_y > -1 * (max_lidar_range-disc_size)):
    pix_x = int(math.floor((pt_x + max_lidar_range) * disc_factor))
    pix_y = int(math.floor((max_lidar_range - pt_y) * disc_factor))
    blank_image[pix_y,pix_x] = [255,0,0]

blank_image[(blank_image == [0, 0, 0]).all(axis=-1)] = [255, 255, 255]

In [8]:
rospy.wait_for_service('/set_map')
map_setter = rospy.ServiceProxy('/set_map', SetMap)

In [10]:
pose_with_convariance_stamped = PoseWithCovarianceStamped()
pose_with_convariance_stamped.header.frame_id = 'map'
pose_with_covariance = PoseWithCovariance()
cpose = Pose()
cpose.position.x = 1.0
cpose.position.y = 0
cpose.position.z = 0
cpose.orientation.x = 0
cpose.orientation.y = 0
cpose.orientation.z = 0
cpose.orientation.w = 1
# cpose 
pose_with_covariance.pose = cpose
pose_with_covariance.covariance = np.array([1,   0,   0,   0,   0,   0,
                               0,   1,   0,   0,   0,   0,
                               0,   0,   1,   0,   0,   0,
                               0,   0,   0,   1,   0,   0,
                               0,   0,   0,   0,   1,   0,
                               0,   0,   0,   0,   0,   .7])**2
pose_with_convariance_stamped.pose = pose_with_covariance
map_setter(occupancy_grid, pose_with_convariance_stamped)

success: True

In [ ]:
%matplotlib inline 
from matplotlib import pyplot as plt
plt.figure(figsize=(12,12))
plt.imshow(blank_image)
plt.show()

In [ ]:
# find the table first
rospy.wait_for_service('table_searcher/search_table')
table_searcher = rospy.ServiceProxy('table_searcher/search_table', SearchTable)

In [ ]:
table_info = table_searcher()

In [ ]:
table_pc = convert_pointcloud2_to_pc(table_info.point_cloud)

In [ ]:
# Loop through all points plot in blank_image
for table_point in table_pc:
    pt_x = table_point[0]
    pt_y = table_point[1]
    if (pt_x < max_lidar_range) or (pt_x > -1 * (max_lidar_range-disc_size)) or (pt_y < max_lidar_range) or (pt_y > -1 * (max_lidar_range-disc_size)):
        pix_x = int(math.floor((pt_x + max_lidar_range) * disc_factor))
        pix_y = int(math.floor((max_lidar_range - pt_y) * disc_factor))
        if (pix_x > image_size) or (pix_y > image_size):
            print "Error"
        elif not np.array_equal(blank_image[pix_y,pix_x], [0, 0, 255]):
            blank_image[pix_y,pix_x] = [0,255,255]

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(blank_image)
plt.show()

In [ ]:
# search for the object
rospy.wait_for_service('table_searcher/segment_objects')
object_searcher = rospy.ServiceProxy('table_searcher/segment_objects', SegmentObjects)

In [ ]:
detected_objects = object_searcher()
print("detected objects' number: ", len(detected_objects.segmented_objects.objects))

In [ ]:
object_pc = convert_pointcloud2_to_pc(detected_objects.segmented_objects.objects[0].point_cloud)
object_mean_point = np.mean(object_pc, axis=0)
print "object mean point ", object_mean_point

In [ ]:
# Loop through all points plot in blank_image
for object_point in object_pc:
    pt_x = object_point[0]
    pt_y = object_point[1]
    if (pt_x < max_lidar_range) or (pt_x > -1 * (max_lidar_range-disc_size)) or (pt_y < max_lidar_range) or (pt_y > -1 * (max_lidar_range-disc_size)):
        pix_x = int(math.floor((pt_x + max_lidar_range) * disc_factor))
        pix_y = int(math.floor((max_lidar_range - pt_y) * disc_factor))
        if (pix_x > image_size) or (pix_y > image_size):
            print "Error"
        elif not np.array_equal(blank_image[pix_y,pix_x], [0, 0, 255]):
            blank_image[pix_y,pix_x] = [0,255,0]
            
            

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(blank_image)
plt.show()

In [ ]:
# interpolate from base to object point
step_size = 0.1
step = (object_mean_point[:2] / np.linalg.norm(object_mean_point[:2])) * step_size
print step

In [ ]:
current_point = np.array([0.0, 0.0])
robot_size = int(math.floor(0.3 * disc_factor))
stand_position = np.array([0.0,0.0])
while np.linalg.norm(object_mean_point[:2] - current_point) > (step_size * 1.2):
    current_point += step
    
    # show the robot point
    pt_x = current_point[0]
    pt_y = current_point[1]
    pix_x = int(math.floor((pt_x + max_lidar_range) * disc_factor))
    pix_y = int(math.floor((max_lidar_range - pt_y) * disc_factor))
    
    current_image = blank_image[pix_y - robot_size: pix_y + robot_size, pix_x - robot_size: pix_x + robot_size]
    if np.any(np.all(np.array([0, 255, 255]) == current_image, axis=-1)):
        # meet the table
        stand_position[0] = current_point[0]
        stand_position[1] = current_point[1]
        break
        
print "stand position for manipulation"
print stand_position
        
pix_x = int(math.floor((stand_position[0] + max_lidar_range) * disc_factor))
pix_y = int(math.floor((max_lidar_range - stand_position[1]) * disc_factor))
blank_image[pix_y,pix_x] = [0,0,0]


In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(blank_image)
plt.show()